In [4]:
import itertools
import pandas as pd


def generate_score_ballots_SF():
    """
    District	Candidate	Votes	Political Leaning
    5	DEAN PRESTON	21,431	Far-left
    9	HILLARY RONEN	18,335	Far-left
    1	CONNIE CHAN	13,422	Far-left
    11	JOHN AVALOS	13,335	Far-left
    10	SHAMANN WALTON	9,550	Far-left
    7	VILASKA NGUYEN	8,195	Far-left
    10	TONY KELLY	5,643	Far-left
    7	EMILY MURASE	4,851	Far-left
    3	AARON PESKIN	15,293	Left
    8	MATT HANEY	14,249	Left
    4	GORDON MAR	10,288	Left
    7	MYRNA MELGAR	7,852	Left
    1	DAVID E. LEE	6,071	Left
    5	VALLIE BROWN	16,730	Moderate
    11	AHSHA SAFAI	15,033	Moderate
    2	CATHERINE STEFANI	14,378	Moderate
    2	NICK JOSEFOWITZ	13,617	Moderate
    1	MARJAN PHILHOUR	12,197	Moderate
    3	DANNY SAUTER	10,451	Moderate
    9	JOSHUA ARCE	9,612	Moderate
    7	JOEL ENGARDIO	9,216	Moderate
    4	JESSICA HO	7,423	Moderate
    8	CHRISTINE JOHNSON	6,237	Moderate
    10	THEO ELLINGTON	4,800	Moderate
    8	SONJA TRAUSS	4,759	Moderate
    2	SCHUYLER HUDAK	4,132	Moderate
    4	TREVOR MCNEIL	3,479	Moderate
    7	STEPHEN W. MARTIN-PINTO	4,562	Right

    Current BOS:
    District	Name	Political Leaning
    1	Chan	Far-left
    3	Peskin	Far-left
    4	Mar	Far-left
    5	Preston	Far-left
    9	Ronen	Far-left
    10	Walton	Far-left
    6	Haney	Left
    7	Melgar	Left
    2	Stefani	Moderate
    8	Mandelman	Moderate
    11	Safai	Moderate

    Hypothetical BOS using Allocated Score
    Name	Political Leaning
    DEAN PRESTON	Far-left
    HILLARY RONEN	Far-left
    CONNIE CHAN	Far-left
    JOHN AVALOSFar-left
    AARON PESKIN	Left
    MATT HANEY	Left
    VALLIE BROWN	Moderate
    AHSHA SAFAI	Moderate
    CATHERINE STEFANI	Moderate
    NICK JOSEFOWITZ	Moderate
    MARJAN PHILHOUR	Moderate
    """
    candidates = [
        "DEAN PRESTON",
        "HILLARY RONEN",
        "CONNIE CHAN",
        "JOHN AVALOS",
        "SHAMANN WALTON",
        "VILASKA NGUYEN",
        "TONY KELLY",
        "EMILY MURASE",
        "AARON PESKIN",
        "MATT HANEY",
        "GORDON MAR",
        "MYRNA MELGAR",
        "DAVID E. LEE",
        "VALLIE BROWN",
        "AHSHA SAFAI",
        "CATHERINE STEFANI",
        "NICK JOSEFOWITZ",
        "MARJAN PHILHOUR",
        "DANNY SAUTER",
        "JOSHUA ARCE",
        "JOEL ENGARDIO",
        "JESSICA HO",
        "CHRISTINE JOHNSON",
        "THEO ELLINGTON",
        "SONJA TRAUSS",
        "SCHUYLER HUDAK",
        "TREVOR MCNEIL",
        "STEPHEN W. MARTIN-PINTO",
    ]

    far_left = {
        candidates[0]: 5,
        candidates[1]: 4,
        candidates[2]: 4,
        candidates[3]: 4,
        candidates[4]: 3,
        candidates[5]: 3,
        candidates[6]: 3,
        candidates[7]: 3,
        candidates[8]: 3,
    }

    left = {
        candidates[8]: 5,
        candidates[9]: 4,
        candidates[10]: 3,
        candidates[11]: 3,
        candidates[12]: 3,
    }

    moderate = {
        candidates[13]: 5,
        candidates[14]: 4,
        candidates[15]: 4,
        candidates[16]: 3,
        candidates[17]: 3,
        candidates[18]: 3,
        candidates[19]: 3,
        candidates[20]: 3,
        candidates[21]: 3,
        candidates[22]: 3,
        candidates[23]: 3,
        candidates[24]: 3,
        candidates[25]: 3,
        candidates[26]: 3,
    }

    right = {
        candidates[27]: 5,
    }

    ballots = (
        list(itertools.repeat(far_left, 94762))
        + list(itertools.repeat(left, 53753))
        + list(itertools.repeat(moderate, 132064))
        + list(itertools.repeat(right, 4562))
    )
    ballots = pd.DataFrame(ballots)
    return ballots

In [5]:
import pandas as pd
import numpy as np

# Copied from https://electowiki.org/wiki/Allocated_Score


def Allocated_Score(K, W, S):

    # Normalize score matrix
    ballots = pd.DataFrame(S.values / K, columns=S.columns)

    # Find number of voters and quota size
    V = ballots.shape[0]
    quota = V / W
    ballot_weight = pd.Series(np.ones(V), name="weights")

    # Populate winners in a loop
    winner_list = []
    while len(winner_list) < W:

        weighted_scores = ballots.multiply(ballot_weight, axis="index")

        # Select winner
        w = weighted_scores.sum().idxmax()

        # Add winner to list
        winner_list.append(w)

        # remove winner from ballot
        ballots.drop(w, axis=1, inplace=True)

        # Create lists for manipulation
        cand_df = pd.concat([ballot_weight, weighted_scores[w]], axis=1).copy()
        cand_df_sort = cand_df.sort_values(by=[w], ascending=False).copy()

        # find the score where a quota is filled
        split_point = cand_df_sort[cand_df_sort["weights"].cumsum() < quota][w].min()

        # Amount of ballot for voters who voted more than the split point
        spent_above = cand_df[cand_df[w] > split_point]["weights"].sum()

        # Allocate all ballots above split point
        if spent_above > 0:
            cand_df.loc[cand_df[w] > split_point, "weights"] = 0.0

        # Amount of ballot for voters who gave a score on the split point
        weight_on_split = cand_df[cand_df[w] == split_point]["weights"].sum()

        # Fraction of ballot on split needed to be spent
        if weight_on_split > 0:
            spent_value = (quota - spent_above) / weight_on_split

            # Take the spent value from the voters on the threshold evenly
            cand_df.loc[cand_df[w] == split_point, "weights"] = cand_df.loc[
                cand_df[w] == split_point, "weights"
            ] * (1 - spent_value)

        ballot_weight = cand_df["weights"].clip(0.0, 1.0)

    return winner_list

In [7]:
ballots = generate_score_ballots_SF()
Allocated_Score(5, 11, ballots)

['VALLIE BROWN',
 'AARON PESKIN',
 'DEAN PRESTON',
 'AHSHA SAFAI',
 'CATHERINE STEFANI',
 'HILLARY RONEN',
 'CONNIE CHAN',
 'NICK JOSEFOWITZ',
 'MATT HANEY',
 'MARJAN PHILHOUR',
 'JOHN AVALOS']